In [ ]:
import tushare as ts
import baostock as bs
import pandas as pd
import numpy as np
import time
import re
from datetime import datetime, timedelta
import random
import pyecharts.options as opts
from pyecharts.charts import Line, Bar, Grid,Candlestick,Kline,Page,EffectScatter
from pyecharts.globals import CurrentConfig, NotebookType, SymbolType
from pyecharts.commons.utils import JsCode
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
import CLbasementtmp

In [ ]:
ts.set_token('2e7adb11584089e1560db3041ce85f5818bc138750c95b0a0f67e284')
pro = ts.pro_api()

In [ ]:
#基于tushare获取指定日期数据
def get_stock_day_data(tradedate):
    df = pro.daily(**{
        "ts_code": "",
        "trade_date": tradedate,
        "start_date": "",
        "end_date": "",
        "offset": "",
        "limit": ""
    }, fields=[
        "ts_code",
        "trade_date",
        "open",
        "high",
        "low",
        "close",
        "vol",
        "amount",
        "pct_chg"
    ])
    return df

##基于tushare获取指定日期区间的日历
def get_toshare_trade_cal(startdate, enddate):
    trade_date_df = pro.trade_cal(**{
        "exchange": "",
        "cal_date": "",
        "start_date": startdate,
        "end_date": enddate,
        "is_open": "",
        "limit": "",
        "offset": ""
    }, fields=[
        "exchange",
        "cal_date",
        "is_open",
        "pretrade_date"
    ])
    trade_date_list = trade_date_df[trade_date_df['is_open'] == 1]['cal_date'].to_list()

    
    return trade_date_list

##基于tushare获取指定月份到现在的日历
def get_toshare_day_date(curdate, x):
    # 获取当前日期
    current_date = datetime.strptime(curdate, "%Y-%m-%d" )
    
    # 计算前 4 个月的日期
    # 首先获取当前年份和月份
    year = current_date.year
    month = current_date.month
    
    # 计算前 4 个月的年份和月份
    new_month = month - x
    new_year = year
    if new_month <= 0:
        new_month += 12
        new_year -= 1
    
    # 构造前 4 个月的日期
    # 假设日期保持不变（即当前日期的天数）
    day = current_date.day
    try:
        target_date = datetime(new_year, new_month, day)
    except ValueError:
        # 如果当前日期的天数在目标月份中不存在（如 31 号在 2 月）
        # 则将日期设置为该月的最后一天
        from calendar import monthrange
        day = monthrange(new_year, new_month)[1]
        target_date = datetime(new_year, new_month, day)

    startdate = target_date.strftime("%Y%m%d")
    enddate = current_date.strftime("%Y%m%d") 
    trade_date_df = pro.trade_cal(**{
        "exchange": "",
        "cal_date": "",
        "start_date": startdate,
        "end_date": enddate,
        "is_open": "",
        "limit": "",
        "offset": ""
    }, fields=[
        "exchange",
        "cal_date",
        "is_open",
        "pretrade_date"
    ])
    trade_date_list = trade_date_df[trade_date_df['is_open'] == 1]['cal_date'].to_list()

    
    return trade_date_list
    

In [ ]:
#构建中枢相关
def gen_zs_pos(start_x,start_y,end_x,end_y):
    tmp_list = []
    #构造起点坐标
    tmp_position_start = {'xAxis': 0, 'yAxis': 0}
    tmp_position_start['xAxis'] = start_x
    tmp_position_start['yAxis'] = start_y
    tmp_list.append(tmp_position_start)
    #构造终点坐标
    tmp_position_end = {'xAxis': 0, 'yAxis': 0}
    tmp_position_end['xAxis'] = end_x
    tmp_position_end['yAxis'] = end_y
    tmp_list.append(tmp_position_end)
    #print(tmp_list)
    #存储完成的中枢坐标
    return tmp_list


def generate_zs_area(draw):
    mark_areas = []

    #基于初始两笔计算zs开始位置和draw方向
    if (draw[1][1] > draw[0][1]) and (draw[2][1] <= draw[0][1]):
    #   |
    #  | |
    # |   |
    #      |
        zs_sx = 0
        zs_ex = 0
        zs_high = draw[1][1]
        zs_low = draw[0][1]
        zs_lenth = 1
        segdir = 0

    elif (draw[1][1] > draw[0][1]) and (draw[2][1] > draw[0][1]):
    #    |
    #   | |
    #  |   |
    # |
        zs_sx = 1
        zs_ex = 1
        zs_high = draw[1][1]
        zs_low = draw[2][1]
        zs_lenth = 1
        segdir = 0

    elif (draw[1][1] < draw[0][1]) and (draw[2][1] > draw[0][1]):
    #      |
    # |   |
    #  | |
    #   |
        zs_sx = 0
        zs_ex = 0
        zs_high = draw[0][1]
        zs_low = draw[1][1]
        zs_lenth = 1
        segdir = 1

    elif (draw[1][1] < draw[0][1]) and (draw[2][1] <= draw[0][1]):
    #|     
    # |   |
    #  | |
    #   |
        zs_sx = 1
        zs_ex = 1
        zs_high = draw[2][1]
        zs_low = draw[1][1]
        zs_lenth = 1
        segdir = 1
    
    for i in range(3,len(draw)):
        curx = draw[i][0]
        v0 = draw[i][1]
        segdir = 1 - segdir
        #向下笔
        if segdir == 0:
            #比zs高 
            if (v0 >= zs_high):
                #zs大于等于3笔:保存中枢并结束
                if (zs_lenth >=3):
                    zs_ex = i - 2
                    mark_areas.append(gen_zs_pos(draw[zs_sx][0],zs_high,draw[zs_ex][0],zs_low))
                    zs_sx = i - 1
                    zs_ex = i - 1
                    zs_high = draw[i-1][1]
                    zs_low  = draw[i][1]
                    zs_lenth = 1
                #ZS小于三笔，直接结束
                else:
                    zs_sx = i - 1
                    zs_ex = i - 1
                    zs_high = draw[i-1][1]
                    zs_low  = draw[i][1]
                    zs_lenth = 1
            #在zs区间内
            elif (v0 < zs_high) and (v0 > zs_low):
                zs_low = v0
                zs_lenth = zs_lenth + 1
            #比ZS低
            elif (v0 <= zs_low):
                zs_lenth = zs_lenth + 1
        #向上笔
        elif segdir == 1:
            #比zs低 
            if (v0 <= zs_low) : 
                #zs大于等于3笔:结束
                if (zs_lenth >=3):
                    zs_ex = i - 2
                    mark_areas.append(gen_zs_pos(draw[zs_sx][0],zs_high,draw[zs_ex][0],zs_low))
                    zs_sx = i - 1
                    zs_ex = i - 1
                    zs_high = draw[i][1]
                    zs_low  = draw[i-1][1]
                    zs_lenth = 1
                #ZS小于三笔，直接结束
                else:
                    zs_sx = i - 1
                    zs_ex = i - 1
                    zs_high = draw[i][1]
                    zs_low  = draw[i-1][1]
                    zs_lenth = 1
            #在zs区间内
            elif (v0 < zs_high) and (v0 > zs_low):
                zs_high = v0
                zs_lenth = zs_lenth + 1
            elif (v0 < zs_low):
                zs_lenth = zs_lenth + 1

    if zs_sx < len(draw) - 3:
        zs_ex = len(draw) - 2
        mark_areas.append(gen_zs_pos(draw[zs_sx][0],zs_high,draw[zs_ex][0],zs_low))

    
    return mark_areas

In [ ]:
#作图相关
#重整kindledata
def generate_kindledata(datas):
    res = []
    for index, raw in datas.iterrows():
        res.append([raw['open'], raw['close'],raw['low'],raw['high']])
    return res
#生成蜡烛图
def generate_kindle_chart(kindledata,date_data):
    kindle_line = Kline()
    kindle_line.add_xaxis(date_data)
    kindle_line.add_yaxis("Stock Trend", kindledata)
    kindle_line.set_global_opts(
                title_opts=opts.TitleOpts(title="Stock Price"),
                xaxis_opts=opts.AxisOpts(type_="category", is_scale=True),
                yaxis_opts=opts.AxisOpts(is_scale=True),
                tooltip_opts=opts.TooltipOpts(trigger="axis"),  # 鼠标悬停显示浮窗
                datazoom_opts=[
                    opts.DataZoomOpts(range_start = 50, range_end = 100, type_="inside", xaxis_index=[0, 1, 2]),  # 同步到两个图表
                    opts.DataZoomOpts(range_start = 50, range_end = 100, type_="slider", xaxis_index=[0, 1, 2])   # 同步到两个图表
                ]
    )
    return kindle_line

#生成成交量图
def generate_volbar_chart(voldata, date_data):
    vol_bar = Bar()
    vol_bar.add_xaxis(date_data)
    vol_bar.add_yaxis(
        "vol", voldata, yaxis_index=1, color="green", label_opts=opts.LabelOpts(is_show=False),
                    itemstyle_opts=opts.ItemStyleOpts(
                        color=JsCode(
                            """
                                function(params) {
                                    var colorList;
                                    colorList = '#4EA9F0';
                                    return colorList;
                                }
                                """))  
                     )
    vol_bar.set_global_opts(
            xaxis_opts=opts.AxisOpts(type_="category", is_scale=True,axislabel_opts=opts.LabelOpts(is_show=False)),
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitline_opts=opts.SplitLineOpts(is_show=True),
                position="right",
                axislabel_opts=opts.LabelOpts(is_show=False)  # 隐藏Y轴刻度
            ),
            datazoom_opts=[
                opts.DataZoomOpts(range_start = 50, range_end = 100, type_="inside", xaxis_index=[0, 1, 2]),  # 同步到两个图表
                opts.DataZoomOpts(range_start = 50, range_end = 100, type_="slider", xaxis_index=[0, 1, 2])   # 同步到两个图表
            ],
            legend_opts=opts.LegendOpts(is_show=False),  # 隐藏图例
        )
    return vol_bar

#生成MACD图
def generate_macdbar_chart(macd_data,dif,dea,date_data):
    macd_bar = Bar()
    macd_bar.add_xaxis(date_data)
    macd_bar.add_yaxis(
        "macd", macd_data, yaxis_index=1, color="green", label_opts=opts.LabelOpts(is_show=False),
        itemstyle_opts=opts.ItemStyleOpts(
                        color=JsCode(
                            """
                                function(params) {
                                    var colorList;
                                    if (params.data >= 0) {
                                      colorList = '#ef232a';
                                    } else {
                                      colorList = '#14b143';
                                    }
                                    return colorList;
                                }
                                """))        
    )
    macd_bar.set_global_opts(
            xaxis_opts=opts.AxisOpts(type_="category", is_scale=True,axislabel_opts=opts.LabelOpts(is_show=False)),
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitline_opts=opts.SplitLineOpts(is_show=True),
                position="right",
                axislabel_opts=opts.LabelOpts(is_show=False)  # 隐藏Y轴刻度
            ),
            datazoom_opts=[
                opts.DataZoomOpts(range_start = 50, range_end = 100,type_="inside", xaxis_index=[0, 1, 2]),  # 同步到两个图表
                opts.DataZoomOpts(range_start = 50, range_end = 100,type_="slider", xaxis_index=[0, 1, 2])   # 同步到两个图表
            ],
            legend_opts=opts.LegendOpts(is_show=False),  # 隐藏图例
        ) 
    line = Line()
    line.add_xaxis(date_data)
    line.add_yaxis("dif", dif, yaxis_index=1, color="orange", symbol="none", label_opts=opts.LabelOpts(is_show=False),is_smooth=True)  
    line.add_yaxis("dea", dea, yaxis_index=1, color="black", symbol="none", label_opts=opts.LabelOpts(is_show=False),is_smooth=True)
    macd_bar.overlap(line)
    
    return macd_bar

# 生成ma图
def generate_ma_chart(date_data,ma5, ma10):
    line = Line()
    line.add_xaxis(date_data)
    line.add_yaxis("ma5", ma5, yaxis_index=1, color="black",symbol="none",label_opts=opts.LabelOpts(is_show=False),is_smooth=True)  
    line.add_yaxis("ma10", ma10, yaxis_index=1, color="orange",symbol="none", label_opts=opts.LabelOpts(is_show=False),is_smooth=True)
    return line

#生成折线图：
def generate_draw_chart(draw_data,datelist):
    #添加折线数据
    draw_xaxis = []
    draw_yaxis = []
    for each in draw_data:
        draw_xaxis.append(datelist[each[0]])
        draw_yaxis.append(each[1])
    line_chart = Line()
    line_chart.add_xaxis(xaxis_data=draw_xaxis)
    line_chart.add_yaxis(series_name='draw', y_axis=draw_yaxis,label_opts=opts.LabelOpts(is_show=False))
    line_chart.set_series_opts(
        linestyle_opts=opts.LineStyleOpts(width=2, color='blue' ),  # 设置折线宽度为 2
    )

    return line_chart

def generate_line_chart_square_area(line_data,name,colors,markarea):
    xaxis = []
    yaxis = []
    for each in line_data:
        xaxis.append(each[0])
        yaxis.append(each[1])

    line_chart = Line()
    line_chart.add_xaxis(xaxis_data=xaxis)
    line_chart.add_yaxis(series_name=name, y_axis=yaxis)
    line_chart.set_series_opts(
        linestyle_opts=opts.LineStyleOpts(width=2, color=colors),  # 设置折线宽度为 2
        markarea_opts=opts.MarkAreaOpts(is_silent=True, data=markarea,itemstyle_opts=opts.ItemStyleOpts(color=colors,opacity=0.5))
    )
    return line_chart

# 第三版算法

In [ ]:
#获取股票原始数据
trade_date_list = get_toshare_day_date(curdate, 4)
stock_data = pd.DataFrame()
for tradedate in trade_date_list:
    tmpdf = get_stock_day_data(tradedate)
    stock_data = pd.concat([stock_data, tmpdf], axis=0, ignore_index=True)
    time.sleep(1)

## fliter

In [141]:
#基于近两天平均涨跌幅 > 前10天平均涨跌幅 且近两天平均涨跌幅 > 2
#获取题材信息
curdate = '2025-07-11'
datelist = get_toshare_day_date(curdate, 2)
df_dc_tc_detail = pd.DataFrame()
for each in datelist:
    df_tmp = pro.dc_daily(trade_date=each)
    df_tc_tmp = pro.dc_index(trade_date=each, fields='ts_code,name')
    df_tmp = pd.merge(df_tmp, df_tc_tmp, on='ts_code', how='left')
    df_dc_tc_detail = pd.concat([df_dc_tc_detail, df_tmp],ignore_index=True)
    time.sleep(1)
#题材信息数据清洗
df_dc_tc_detail = df_dc_tc_detail[~df_dc_tc_detail['name'].str.contains('昨日', na=False)]
df_dc_tc_detail = df_dc_tc_detail.sort_values(by='trade_date')

In [142]:
#题材code列表
tc_list = list(set(test_df_tc_detail['ts_code'].tolist()))
tc_trend = pd.DataFrame(columns = ['tc', 'his_avg_c%', 'cur_avg_c%'])
for each in tc_list:
    single_tc_detail = df_dc_tc_detail[df_dc_tc_detail['ts_code'] == each]
    if len(single_tc_detail) >= 12:
        tmpraw = single_tc_detail.tail(12)['pct_change'].to_list()
        hisavg = sum(tmpraw[:10])/10
        curavg = sum(tmpraw[-2:])/2
        if (curavg > hisavg) and (curavg > 2):
            tc_trend.loc[len(tc_trend)] = [each,hisavg,curavg]
    elif (len(single_tc_detail) >= 3) and (len(single_tc_detail) < 12):
        tmpraw = single_tc_detail['pct_change'].to_list()
        hisavg = sum(tmpraw[:len(tmpraw)-2])/(len(tmpraw)-2)
        curavg = sum(tmpraw[-2:])/2
        if (curavg > hisavg) and (curavg > 2):
            tc_trend.loc[len(tc_trend)] = [each,hisavg,curavg]

In [155]:
#热点题材股票列表
hot_tc_list = tc_trend['tc'].tolist()
hot_tc_member = pd.DataFrame()
tushare_date = curdate.replace('-','')
for each in hot_tc_list:
    df_tmp = pro.dc_member(trade_date=tushare_date, ts_code=each)
    hot_tc_member = pd.concat([hot_tc_member, df_tmp],ignore_index=True)
    time.sleep(0.2)
stock_list = hot_tc_member['con_code'].tolist()

## 算法

In [161]:
maybe_buylist = []
for stock_name in stock_list:
    #dataframe处理
    tmpdata = stock_data[stock_data['ts_code']==stock_name]
    tmpdata = tmpdata.copy()
    tmpdata['trade_date'] = pd.to_datetime(tmpdata['trade_date'], format='%Y%m%d', errors='coerce')
    tmpdata.columns = ['code','datetime','open','high','low','close','volume','amount','pct_chg']
    tmpdata = tmpdata.sort_values(by='datetime')
    tmpdata = tmpdata.reset_index(drop=True)
    try:
        #基本数据获取
        original_kline, draw = CLbasementtmp.cl_base(tmpdata)
        #markarea = CLbasementtmp.uplevel_zsbase(draw)
        markarea = generate_zs_area(draw)
        kindledata = generate_kindledata(tmpdata)
        vol = tmpdata['volume'].tolist()
        datelist = tmpdata['datetime'].astype(str).tolist()
        # 判断条件
        #中枢起始点
        zs_start = markarea[-1][0]['xAxis']
        zs_end = markarea[-1][1]['xAxis']
        #中枢价格区间
        zs_price_section = original_kline[zs_start:zs_end+1]
        #中枢价格最高点
        zs_price_high = 0
        for each in zs_price_section:
            tmphigh = each[3]
            if tmphigh > zs_price_high:
                zs_price_high = tmphigh
        
        #中枢成交量区间
        zs_vol_section = vol[zs_start:zs_end+1]
        #中枢区间平均成交量
        zs_avg_vol = int(np.mean(zs_vol_section))
        
        #最后一笔价格高点
        last_draw_price_high = original_kline[-1][3]
        #最后一笔成交量高点
        last_draw_vol_high = max(vol[draw[-2][0]:])
        #最后一笔向上
        if draw[-1][1] > draw[-2][1]:
            last_draw_dir = 1
        else:
            last_draw_dir = 0
        
        #判断：
        #最后一笔价格高点大于中枢高点  最后一笔成交量高点大于中枢平均成交量 
        #最后一笔向上 且是最后一个中枢上来的一笔 \
        #最后一笔的结束是最后一根K线
        if (last_draw_price_high > zs_price_high) and (last_draw_vol_high > zs_avg_vol)\
        and (last_draw_dir == 1) and (markarea[-1][1]['xAxis'] == draw[-2][0])\
        and (len(tmpdata)-1 == draw[-1][0]):
            #print(stock_name)
            maybe_buylist.append(stock_name)
    except Exception as e:
        print(stock_name, e)

601211.SH list index out of range
601211.SH list index out of range
